# 📊 Fantasy Premier League (FPL) - Complete Data Analysis & Strategy Tools

## 🎯 **Overview**
This notebook provides comprehensive analysis tools for Fantasy Premier League decision-making, including:
- **Data Exploration & Cleaning** - Understanding the dataset structure
- **Season Performance Analysis** - Player and team cumulative statistics  
- **Strategic Analysis Tools** - Fixture difficulty, player rankings, team strength
- **Actionable FPL Insights** - Real-world applications for transfers and team selection

## 📋 **Table of Contents**
1. [**Data Loading & Overview**](#data-loading--overview)
2. [**Data Cleaning & Processing**](#data-cleaning--processing)  
3. [**Exploratory Data Analysis**](#exploratory-data-analysis)
4. [**Season Statistics Calculation**](#season-statistics-calculation)
5. [**Player Performance Analysis**](#player-performance-analysis)
6. [**Strategic Analysis Tools**](#strategic-analysis-tools)
7. [**Fixture Analysis System**](#fixture-analysis-system)
8. [**Quick Reference & Usage Guide**](#quick-reference--usage-guide)

---

In [69]:
import pandas as pd 
df = pd.read_csv('fpl-data-stats.csv')
df.describe()

,id,element_type,now_cost,selected_by_percent,gameweek,minutes,shots,SoT,SiB,xG,...,defensive_contribution,xGI,npxGI,xP,total_points,PvsxP,touches,penalty_area_touches,carries_final_third,carries_penalty_area
count,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.00000,5073.000000,...,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,5073.000000,2136.000000,2136.000000,5073.000000,5073.000000
mean,363.141731,2.545634,4.996885,2.053262,4.051646,27.152572,0.321112,0.104475,0.21782,0.034299,...,2.080229,0.058427,0.055490,1.239674,1.248374,0.008700,38.167135,1.467228,0.313621,0.122019
std,209.714541,0.834925,1.097142,6.078028,1.991923,37.781671,0.799732,0.369005,0.62894,0.125756,...,3.636650,0.169397,0.158095,2.027410,2.397877,1.430071,24.773459,1.892634,0.825705,0.513898
min,1.000000,1.000000,3.900000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,-2.000000,-3.000000,-11.400000,0.000000,0.000000,0.000000,0.000000
25%,182.000000,2.000000,4.400000,0.100000,2.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000
50%,363.000000,3.000000,4.800000,0.200000,4.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,36.000000,1.000000,0.000000,0.000000
75%,544.000000,3.000000,5.400000,1.000000,6.000000,70.000000,0.000000,0.000000,0.00000,0.000000,...,3.000000,0.000000,0.000000,2.100000,1.000000,0.000000,54.000000,2.000000,0.000000,0.000000
max,743.000000,4.000000,14.500000,61.500000,7.000000,90.000000,7.000000,4.000000,7.00000,2.000000,...,23.000000,2.000000,2.000000,13.000000,24.000000,12.826000,129.000000,18.000000,8.000000,11.000000


# 1️⃣ Data Loading & Overview {#data-loading--overview}

## 📂 Import Data and Initial Exploration
This section loads the FPL dataset and provides basic information about its structure.

In [70]:
# Dataset Overview and Structure
print("=== DATASET OVERVIEW ===")
print(f"Dataset Shape: {df.shape}")
print(f"Total Records: {df.shape[0]:,}")
print(f"Total Features: {df.shape[1]}")
print("\n=== COLUMN NAMES ===")
print(df.columns.tolist())

print("\n=== DATA TYPES ===")
print(df.dtypes)

print("\n=== BASIC INFO ===")
df.info()

=== DATASET OVERVIEW ===
Dataset Shape: (5073, 37)
Total Records: 5,073
Total Features: 37

=== COLUMN NAMES ===
['id', 'element_type', 'web_name', 'team_name', 'opponent_team_name', 'was_home', 'now_cost', 'selected_by_percent', 'gameweek', 'minutes', 'shots', 'SoT', 'SiB', 'xG', 'npxG', 'G', 'npG', 'key_passes', 'xA', 'A', 'xGC', 'GC', 'xCS', 'CS', 'clearances_blocks_interceptions', 'recoveries', 'tackles', 'defensive_contribution', 'xGI', 'npxGI', 'xP', 'total_points', 'PvsxP', 'touches', 'penalty_area_touches', 'carries_final_third', 'carries_penalty_area']

=== DATA TYPES ===
id                                   int64
element_type                         int64
web_name                            object
team_name                           object
opponent_team_name                  object
was_home                              bool
now_cost                           float64
selected_by_percent                float64
gameweek                             int64
minutes                  

In [71]:
# Missing Values Analysis
print("=== MISSING VALUES ANALYSIS ===")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing Count': missing_values.values,
    'Missing Percentage': missing_percentage.values
}).sort_values('Missing Count', ascending=False)

# Display only columns with missing values
if missing_df['Missing Count'].sum() > 0:
    print(missing_df[missing_df['Missing Count'] > 0])
else:
    print("No missing values found in the dataset!")

print(f"\nTotal missing values in dataset: {missing_values.sum():,}")
print(f"Percentage of complete records: {((len(df) - missing_values.sum()) / len(df)) * 100:.2f}%")

df = df.drop(columns=['penalty_area_touches', 'touches'])

=== MISSING VALUES ANALYSIS ===
                  Column  Missing Count  Missing Percentage
34  penalty_area_touches           2937           57.894737
33               touches           2937           57.894737

Total missing values in dataset: 5,874
Percentage of complete records: -15.79%


# 2️⃣ Data Cleaning & Processing {#data-cleaning--processing}

## 🧹 Data Quality Assessment and Cleaning
Analyzing missing values, data types, and performing necessary data cleaning operations.

In [72]:
# Separate Numerical and Categorical Variables
import numpy as np

# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print("=== VARIABLE TYPES ===")
print(f"Numerical variables ({len(numerical_cols)}): {numerical_cols}")
print(f"\nCategorical variables ({len(categorical_cols)}): {categorical_cols}")

# For categorical variables, show unique values
print("\n=== CATEGORICAL VARIABLES ANALYSIS ===")
for col in categorical_cols[:10]:  # Show first 10 categorical columns
    unique_count = df[col].nunique()
    print(f"\n{col}:")
    print(f"  - Unique values: {unique_count}")
    if unique_count <= 20:  # Show values if not too many
        print(f"  - Values: {sorted(df[col].unique())}")
    else:
        print(f"  - Top 10 values: {df[col].value_counts().head(10).index.tolist()}")

=== VARIABLE TYPES ===
Numerical variables (31): ['id', 'element_type', 'now_cost', 'selected_by_percent', 'gameweek', 'minutes', 'shots', 'SoT', 'SiB', 'xG', 'npxG', 'G', 'npG', 'key_passes', 'xA', 'A', 'xGC', 'GC', 'xCS', 'CS', 'clearances_blocks_interceptions', 'recoveries', 'tackles', 'defensive_contribution', 'xGI', 'npxGI', 'xP', 'total_points', 'PvsxP', 'carries_final_third', 'carries_penalty_area']

Categorical variables (3): ['web_name', 'team_name', 'opponent_team_name']

=== CATEGORICAL VARIABLES ANALYSIS ===

web_name:
  - Unique values: 722
  - Top 10 values: ['Patterson', "O'Brien", 'Martinez', 'Roberts', 'Henderson', 'Wilson', 'Barnes', 'King', 'White', 'James']

team_name:
  - Unique values: 20
  - Values: ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds', 'Liverpool', 'Man City', 'Man Utd', 'Newcastle', "Nott'm Forest", 'Spurs', 'Sunderland', 'West Ham', 'Wolves']

opponent_team_name:

In [73]:
# Filter useful numerical variables for FPL analysis
print("=== FILTERING USEFUL NUMERICAL VARIABLES ===")

# Define categories of useful variables
core_performance = ['total_points', 'minutes', 'now_cost', 'selected_by_percent']
attacking_metrics = ['G', 'A', 'xG', 'xA', 'shots', 'SoT', 'key_passes']
expected_metrics = ['xG', 'xA', 'xGI', 'npxG', 'npxGI', 'xP']
defensive_metrics = ['CS', 'xCS', 'GC', 'xGC', 'tackles', 'recoveries', 
                    'clearances_blocks_interceptions', 'defensive_contribution']
advanced_metrics = ['PvsxP', 'carries_final_third', 'carries_penalty_area']

# Combine into useful variables list
useful_numerical_vars = list(set(core_performance + attacking_metrics + 
                                expected_metrics + defensive_metrics + advanced_metrics))

# Filter only variables that exist in the dataset
useful_vars_available = [var for var in useful_numerical_vars if var in numerical_cols]

print(f"Original numerical variables: {len(numerical_cols)}")
print(f"Useful numerical variables: {len(useful_vars_available)}")
print(f"Variables removed: {len(numerical_cols) - len(useful_vars_available)}")

print(f"\n=== USEFUL VARIABLES BY CATEGORY ===")
print(f"Core Performance: {[v for v in core_performance if v in useful_vars_available]}")
print(f"Attacking Metrics: {[v for v in attacking_metrics if v in useful_vars_available]}")
print(f"Expected Stats: {[v for v in expected_metrics if v in useful_vars_available]}")
print(f"Defensive Metrics: {[v for v in defensive_metrics if v in useful_vars_available]}")
print(f"Advanced Metrics: {[v for v in advanced_metrics if v in useful_vars_available]}")

# Variables to exclude (less useful for FPL analysis)
excluded_vars = [var for var in numerical_cols if var not in useful_vars_available]
print(f"\n=== EXCLUDED VARIABLES ===")
print(f"Less useful for FPL: {excluded_vars}")

# Create filtered dataset with useful variables only
useful_numerical_df = df[useful_vars_available].copy()
print(f"\n=== FILTERED DATASET INFO ===")
print(f"Shape: {useful_numerical_df.shape}")
print(f"Useful numerical variables: {useful_vars_available}")

=== FILTERING USEFUL NUMERICAL VARIABLES ===
Original numerical variables: 31
Useful numerical variables: 26
Variables removed: 5

=== USEFUL VARIABLES BY CATEGORY ===
Core Performance: ['total_points', 'minutes', 'now_cost', 'selected_by_percent']
Attacking Metrics: ['G', 'A', 'xG', 'xA', 'shots', 'SoT', 'key_passes']
Expected Stats: ['xG', 'xA', 'xGI', 'npxG', 'npxGI', 'xP']
Defensive Metrics: ['CS', 'xCS', 'GC', 'xGC', 'tackles', 'recoveries', 'clearances_blocks_interceptions', 'defensive_contribution']
Advanced Metrics: ['PvsxP', 'carries_final_third', 'carries_penalty_area']

=== EXCLUDED VARIABLES ===
Less useful for FPL: ['id', 'element_type', 'gameweek', 'SiB', 'npG']

=== FILTERED DATASET INFO ===
Shape: (5073, 26)
Useful numerical variables: ['npxG', 'clearances_blocks_interceptions', 'carries_penalty_area', 'G', 'minutes', 'npxGI', 'carries_final_third', 'defensive_contribution', 'A', 'CS', 'xGC', 'shots', 'selected_by_percent', 'PvsxP', 'recoveries', 'tackles', 'SoT', 'xP',

In [74]:
import pandas as pd
import warnings

# Define team short name mapping
team_short_names = {
    'Liverpool': 'LIV',
    'Man City': 'MCI',
    'Man Utd': 'MUN',
    'Chelsea': 'CHE',
    'Crystal Palace': 'CRY',
    'Bournemouth': 'BOU',
    'Spurs': 'TOT',
    'Everton': 'EVE',
    "Nott'm Forest": 'NFO',
    'Brighton': 'BHA',
    'Newcastle': 'NEW',
    'West Ham': 'WHU',
    'Sunderland': 'SUN',
    'Fulham': 'FUL',
    'Leeds': 'LEE',
    'Aston Villa': 'AVL',
    'Brentford': 'BRE',
    'Wolves': 'WOL',
    'Burnley': 'BUR'
}

def add_team_short_names(season_data: pd.DataFrame) -> pd.DataFrame:
    """
    Add team_name_short column to season_data based on team_name mapping.
    
    Args:
        season_data: DataFrame containing player season statistics
    Returns:
        Updated DataFrame with team_name_short column
    """
    # Create a copy to avoid modifying the original
    season_data = season_data.copy()
    
    # Print unique team names for diagnostics
    unique_teams = season_data['team_name'].unique()
    print("Unique team names in dataframe:", unique_teams)
    
    # Normalize team names for mapping (case-insensitive, strip spaces/punctuation)
    normalized_mapping = {k.lower().replace("'", "").strip(): v for k, v in team_short_names.items()}
    
    # Add team_name_short column with normalized matching
    def map_team_name(team_name):
        if pd.isna(team_name):
            return None
        normalized_name = team_name.lower().replace("'", "").strip()
        return normalized_mapping.get(normalized_name, team_name[:3].upper())  # Default to first 3 letters if unmapped
    
    season_data['team_name_short'] = season_data['team_name'].apply(map_team_name)
    
    # Check for unmapped team names (NaN or defaulted to first 3 letters)
    unmapped_teams = season_data[
        season_data['team_name_short'].isna() | 
        ~season_data['team_name'].str.lower().replace("'", "").str.strip().isin(normalized_mapping.keys())
    ]['team_name'].unique()
    if len(unmapped_teams) > 0:
        warnings.warn(f"Unmapped team names (assigned default short names): {unmapped_teams}. Consider updating the team_short_names mapping.")
    
    return season_data

# Apply the mapping
df = add_team_short_names(df)


Unique team names in dataframe: ['Arsenal' 'Aston Villa' 'Bournemouth' 'Brentford' 'Man Utd' 'Brighton'
 'Man City' 'Burnley' 'Chelsea' 'Everton' 'Sunderland' 'Crystal Palace'
 'Fulham' 'Leeds' 'Liverpool' 'Newcastle' "Nott'm Forest" 'Spurs'
 'West Ham' 'Wolves']


C:\Users\kahme\AppData\Local\Temp\ipykernel_24788\213404417.py:61: UserWarning: Unmapped team names (assigned default short names): ['Arsenal' "Nott'm Forest"]. Consider updating the team_short_names mapping.
  warnings.warn(f"Unmapped team names (assigned default short names): {unmapped_teams}. Consider updating the team_short_names mapping.")


In [75]:
# Display the first 20 rows of the dataset
print("=== TOP 20 ROWS OF DATASET ===")
print(df.head(20))


=== TOP 20 ROWS OF DATASET ===
    id  element_type      web_name team_name opponent_team_name  was_home  \
0    1             1          Raya   Arsenal            Man Utd     False   
1    2             1  Arrizabalaga   Arsenal            Man Utd     False   
2    3             1          Hein   Arsenal            Man Utd     False   
3    4             1       Setford   Arsenal            Man Utd     False   
4    5             2       Gabriel   Arsenal            Man Utd     False   
5    6             2        Saliba   Arsenal            Man Utd     False   
6    7             2     Calafiori   Arsenal            Man Utd     False   
7    8             2      J.Timber   Arsenal            Man Utd     False   
8    9             2        Kiwior   Arsenal            Man Utd     False   
9   10             2  Lewis-Skelly   Arsenal            Man Utd     False   
10  11             2         White   Arsenal            Man Utd     False   
11  12             2     Zinchenko   Arsenal 

In [76]:
# Outlier Detection and Analysis
print("=== OUTLIER DETECTION ===")

def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analyze outliers for key metrics
key_metrics = ['total_points', 'now_cost', 'selected_by_percent', 'minutes']

for metric in key_metrics:
    if metric in df.columns and df[metric].notna().sum() > 0:
        outliers, lower, upper = detect_outliers_iqr(df, metric)
        print(f"\n{metric.upper()}:")
        print(f"  Normal range: {lower:.2f} to {upper:.2f}")
        print(f"  Number of outliers: {len(outliers)}")
        print(f"  Percentage of outliers: {(len(outliers) / len(df)) * 100:.2f}%")
        
        if len(outliers) > 0 and len(outliers) <= 10:
            print("  Top outliers:")
            top_outliers = outliers.nlargest(10, metric)[['web_name', 'team_name', metric]]
            for _, player in top_outliers.iterrows():
                print(f"    {player['web_name']} ({player['team_name']}): {player[metric]}")


=== OUTLIER DETECTION ===

TOTAL_POINTS:
  Normal range: -1.50 to 2.50
  Number of outliers: 800
  Percentage of outliers: 15.77%

NOW_COST:
  Normal range: 2.90 to 6.90
  Number of outliers: 253
  Percentage of outliers: 4.99%

SELECTED_BY_PERCENT:
  Normal range: -1.25 to 2.35
  Number of outliers: 831
  Percentage of outliers: 16.38%

MINUTES:
  Normal range: -105.00 to 175.00
  Number of outliers: 0
  Percentage of outliers: 0.00%


# 3️⃣ Exploratory Data Analysis {#exploratory-data-analysis}

## 🔍 Deep Dive into Data Patterns
Exploring data distributions, outliers, and relationships between variables.

In [77]:
# Positional and Team Analysis
print("=== POSITIONAL ANALYSIS ===")

# Position mapping
position_map = {1: 'Goalkeeper', 2: 'Defender', 3: 'Midfielder', 4: 'Forward'}
df['position_name'] = df['element_type'].map(position_map)

# Analysis by position
position_stats = df.groupby('position_name').agg({
    'total_points': ['count', 'mean', 'median', 'max'],
    'now_cost': ['mean', 'median'],
    'minutes': ['mean'],
    'selected_by_percent': ['mean'],
    'G': ['mean'],
    'A': ['mean']
}).round(2)

print("Position Statistics:")
print(position_stats)

print("\n=== TEAM ANALYSIS ===")

# Team performance analysis
team_stats = df.groupby('team_name').agg({
    'total_points': ['count', 'sum', 'mean'],
    'now_cost': ['mean'],
    'selected_by_percent': ['mean'],
    'G': ['sum'],
    'A': ['sum'],
    'minutes': ['sum']
}).round(2)

team_stats.columns = ['_'.join(col) for col in team_stats.columns]
team_stats = team_stats.sort_values('total_points_sum', ascending=False)

print("\nTop 10 Teams by Total Points:")
print(team_stats.head(10)[['total_points_sum', 'total_points_mean', 'now_cost_mean']])

print("\n=== VALUE ANALYSIS BY POSITION ===")
# Calculate points per million by position
df['points_per_million'] = df['total_points'] / df['now_cost']

value_by_position = df[df['total_points'] > 0].groupby('position_name')['points_per_million'].agg([
    'count', 'mean', 'median', 'max'
]).round(2)

print(value_by_position)



=== POSITIONAL ANALYSIS ===
Position Statistics:
              total_points                  now_cost        minutes  \
                     count  mean median max     mean median    mean   
position_name                                                         
Defender              1677  1.35    0.0  24     4.49    4.4   31.05   
Forward                550  1.24    0.0  16     5.79    5.4   22.22   
Goalkeeper             589  0.82    0.0  15     4.32    4.0   21.39   
Midfielder            2257  1.29    0.0  18     5.36    5.0   26.96   

              selected_by_percent     G     A  
                             mean  mean  mean  
position_name                                  
Defender                     2.09  0.01  0.02  
Forward                      3.76  0.10  0.02  
Goalkeeper                   2.33  0.00  0.00  
Midfielder                   1.54  0.04  0.04  

=== TEAM ANALYSIS ===

Top 10 Teams by Total Points:
                total_points_sum  total_points_mean  now_cost_m

# 5️⃣ Player Performance Analysis {#player-performance-analysis}

## 🏆 Feature 1 Season Leaders, Value Picks & Hidden Gems
Analysis of top performers using **cumulative season statistics** (not single gameweek data).

In [78]:
# Calculate cumulative season statistics for each player
print("=== CALCULATING CUMULATIVE SEASON STATISTICS ===")

# Group by player and calculate season totals
season_stats = df.groupby(['web_name', 'team_name','team_name_short',  'element_type', 'now_cost', 'selected_by_percent']).agg({
    'total_points': 'sum',  # Sum of all gameweek points
    'minutes': 'sum',       # Total minutes played
    'G': 'sum',            # Total goals
    'A': 'sum',            # Total assists  
    'xG': 'sum',           # Total expected goals
    'xA': 'sum',           # Total expected assists
    'shots': 'sum',        # Total shots
    'SoT': 'sum',          # Total shots on target
    'key_passes': 'sum',   # Total key passes
    'CS': 'sum',           # Total clean sheets
    'xCS': 'sum',          # Total expected clean sheets
    'GC': 'sum',           # Total goals conceded
    'xGC': 'sum',          # Total expected goals conceded
    'gameweek': ['count', 'max'],  # Games played and latest gameweek
    'SiB': 'sum',          # Total shots in box
    'tackles': 'sum',      # Total tackles
    'recoveries': 'sum',    # Total recoveries
    'clearances_blocks_interceptions' : 'sum',
    'defensive_contribution' : 'sum'

}).round(2)

print("Columns after aggregation:")
print(season_stats.columns.tolist())

# Flatten column names
season_stats.columns = ['_'.join(col) if col[1] else col[0] for col in season_stats.columns]
season_stats = season_stats.rename(columns={
    'gameweek_count': 'games_played',
    'gameweek_max': 'last_gameweek'
})

print("Columns after flattening:")
print(season_stats.columns.tolist())

# Reset index to make it a regular dataframe
season_stats = season_stats.reset_index()

# Add position names
position_map = {1: 'Goalkeeper', 2: 'Defender', 3: 'Midfielder', 4: 'Forward'}
season_stats['position_name'] = season_stats['element_type'].map(position_map)

# Calculate additional metrics using the correct column names
season_stats['points_per_million'] = season_stats['total_points_sum'] / season_stats['now_cost']
season_stats['points_per_game'] = season_stats['total_points_sum'] / season_stats['games_played']
season_stats['minutes_per_game'] = season_stats['minutes_sum'] / season_stats['games_played']
season_stats['goals_per_game'] = season_stats['G_sum'] / season_stats['games_played']
season_stats['assists_per_game'] = season_stats['A_sum'] / season_stats['games_played']

# Rename main columns for clarity
season_stats = season_stats.rename(columns={
    'total_points_sum': 'season_points',
    'minutes_sum': 'season_minutes',
    'G_sum': 'season_goals',
    'A_sum': 'season_assists',
    'xG_sum': 'season_xG',
    'xA_sum': 'season_xA',
    'shots_sum': 'season_shots',
    'SoT_sum': 'season_SoT',
    'key_passes_sum': 'season_key_passes',
    'CS_sum': 'season_CS',
    'xCS_sum': 'season_xCS',
    'GC_sum': 'season_GC',
    'xGC_sum': 'season_xGC',
    'SiB_sum': 'season_SiB',
    'tackles_sum': 'season_tackles',
    'recoveries_sum': 'season_recoveries',
    'clearances_blocks_interceptions': 'season_clearances_blocks_interceptions',
    'defensive_contribution' : 'season_defensive_contribution'
    
})

# Round all numeric columns
numeric_cols = season_stats.select_dtypes(include=[np.number]).columns
season_stats[numeric_cols] = season_stats[numeric_cols].round(2)

print(f"Created season stats for {len(season_stats)} players")
print(f"Data covers gameweeks 1-{df['gameweek'].max()}")
season_stats.head(4)

=== CALCULATING CUMULATIVE SEASON STATISTICS ===
Columns after aggregation:
[('total_points', 'sum'), ('minutes', 'sum'), ('G', 'sum'), ('A', 'sum'), ('xG', 'sum'), ('xA', 'sum'), ('shots', 'sum'), ('SoT', 'sum'), ('key_passes', 'sum'), ('CS', 'sum'), ('xCS', 'sum'), ('GC', 'sum'), ('xGC', 'sum'), ('gameweek', 'count'), ('gameweek', 'max'), ('SiB', 'sum'), ('tackles', 'sum'), ('recoveries', 'sum'), ('clearances_blocks_interceptions', 'sum'), ('defensive_contribution', 'sum')]
Columns after flattening:
['total_points_sum', 'minutes_sum', 'G_sum', 'A_sum', 'xG_sum', 'xA_sum', 'shots_sum', 'SoT_sum', 'key_passes_sum', 'CS_sum', 'xCS_sum', 'GC_sum', 'xGC_sum', 'games_played', 'last_gameweek', 'SiB_sum', 'tackles_sum', 'recoveries_sum', 'clearances_blocks_interceptions_sum', 'defensive_contribution_sum']
Created season stats for 759 players
Data covers gameweeks 1-7


,web_name,team_name,team_name_short,element_type,now_cost,selected_by_percent,season_points,season_minutes,season_goals,season_assists,...,season_tackles,season_recoveries,clearances_blocks_interceptions_sum,defensive_contribution_sum,position_name,points_per_million,points_per_game,minutes_per_game,goals_per_game,assists_per_game
0,A.Becker,Liverpool,LIV,1,5.4,8.0,20,540,0.0,0.0,...,0,51,12,0,Goalkeeper,3.70,2.86,77.14,0.0,0.0
1,A.García,Aston Villa,AVL,2,3.9,0.3,0,0,0.0,0.0,...,0,0,0,0,Defender,0.00,0.00,0.00,0.0,0.0
2,A.Jimenez,Bournemouth,BOU,2,4.5,0.0,1,149,0.0,0.0,...,7,7,4,11,Defender,0.22,0.25,37.25,0.0,0.0
3,A.Murphy,Newcastle,NEW,2,3.9,0.7,0,0,0.0,0.0,...,0,0,0,0,Defender,0.00,0.00,0.00,0.0,0.0


In [79]:
# Calculate the required metrics
num_players = season_stats['web_name'].nunique()
total_teams = season_stats['team_name'].nunique()
total_gameweeks = season_stats['last_gameweek'].max()

# Create a summary DataFrame
layout_df = pd.DataFrame({
    'number_of_players': [num_players],
    'total_teams': [total_teams],
    'total_gameweeks': [total_gameweeks]
})

print("Layout Data:")
print(layout_df)
# Export to JSON file
layout_df.to_json('data/layout.json', orient='records', indent=4)

print("Layout data exported to data/layout.json")

Layout Data:
   number_of_players  total_teams  total_gameweeks
0                722           20                7
Layout data exported to data/layout.json


In [ ]:
# Top Performers
print("🏆 === FPL KEY INSIGHTS & RECOMMENDATIONS ===")

# First, calculate form for all players (last 3 gameweeks performance)
def calculate_player_form(player_name, team_name):
    """Calculate form as points per game from recent performances"""
    player_games = df[(df['web_name'] == player_name) & (df['team_name'] == team_name)]
    if len(player_games) == 0:
        print(f"Warning: No data for {player_name} ({team_name})")
        return None  # Changed from 5.0 to None for missing data
    
    # Get last 3 gameweeks or all available games
    recent_games = player_games.nlargest(3, 'gameweek')
    if len(recent_games) == 0:
        print(f"Warning: No recent games for {player_name} ({team_name})")
        return None  # Changed from 5.0 to None
    
    avg_points = recent_games['total_points'].mean()
    element_type = player_games['element_type'].iloc[0] if 'element_type' in player_games else 3  # Default to MID if missing
    
    if element_type == 1:  # Goalkeeper
        form_score = min(10.0, max(0.0, avg_points * 1.2))
    elif element_type == 2:  # Defender
        form_score = min(10.0, max(0.0, avg_points * 1.1))
    else:  # Midfielder or Forward
        form_score = min(10.0, max(0.0, avg_points * 0.9))
    
    return round(form_score, 1)

# Add form to season_stats
season_stats['form'] = season_stats.apply(
    lambda row: calculate_player_form(row['web_name'], row['team_name']), axis=1
)

# Prepare data structures for JSON export
insights_data = {}

# 1. TOP POINT SCORERS (Season Total) - Most reliable performers
print("\n⭐ TOP 15 SEASON PERFORMERS")
print("-" * 50)
top_scorers = season_stats.nlargest(15, 'season_points')

season_performers_data = []
for i, (_, player) in enumerate(top_scorers.iterrows(), 1):
    ppg = player['season_points'] / player['games_played'] if player['games_played'] > 0 else 0
    
    player_data = {
        "player": player['web_name'],
        "team": player['team_name'],
        "team_short": player['team_name_short'],
        "position": player['position_name'],
        "points": int(player['season_points']),
        "ppg": round(ppg, 1),
        "price": player['now_cost'],
        "ownership": player['selected_by_percent'],
        "form": player['form']
    }
    season_performers_data.append(player_data)
    
    print(f"{i:2d}. {player['web_name']} ({player['position_name']}, {player['team_name']} [{player['team_name_short']}])")
    print(f"    {player['season_points']:.0f} pts ({ppg:.1f} ppg) | £{player['now_cost']}m | {player['selected_by_percent']}% owned | Form: {player['form']}")

insights_data['season_performers'] = season_performers_data

# 2. BEST VALUE PICKS - Points per million
print(f"\n💰 BEST VALUE PLAYERS (£/Points Efficiency)")
print("-" * 50)
top_scorer_names = set(top_scorers['web_name'])
value_candidates = season_stats[
    (season_stats['season_points'] >= 15) & 
    (season_stats['points_per_million'] > 0) &
    (~season_stats['web_name'].isin(top_scorer_names))
]
value_players = value_candidates.nlargest(12, 'points_per_million')

value_players_data = []
for i, (_, player) in enumerate(value_players.iterrows(), 1):
    player_data = {
        "player": player['web_name'],
        "team": player['team_name'],
        "team_short": player['team_name_short'],
        "position": player['position_name'],
        "pointsPerMillion": round(player['points_per_million'], 2),
        "totalPoints": int(player['season_points']),
        "price": player['now_cost'],
        "form": player['form']
    }
    value_players_data.append(player_data)
    
    print(f"{i}. {player['web_name']} ({player['position_name']}, {player['team_name']} [{player['team_name_short']}])")
    print(f"   {player['points_per_million']:.2f} pts/£m | {player['season_points']:.0f} pts | £{player['now_cost']}m | Form: {player['form']}")

insights_data['value_players'] = value_players_data

# 3. HIDDEN GEMS - Low ownership with strong underlying stats
print(f"\n💎 HIDDEN GEMS (Low Ownership + Strong Potential)")
print("-" * 50)

# Compute dynamic thresholds based on averages
avg_points = season_stats['season_points'].mean()
avg_form = season_stats['form'].mean()
min_games = 4  # Minimum games played, reasonable for Gameweek 6
min_xG = season_stats['season_xG'].mean() * 0.8  # 80% of average xG for attacking threat

print(f"Dynamic thresholds: Avg Points = {avg_points:.2f}, Avg Form = {avg_form:.2f}, Min xG = {min_xG:.2f}")

# Filter hidden gems using dynamic thresholds
hidden_gems = season_stats[
    (season_stats['season_points'] >= avg_points * 0.8) &  # 80% of average points
    (season_stats['selected_by_percent'] < 8) &
    (season_stats['selected_by_percent'] > 0) &
    (season_stats['games_played'] >= min_games) &
    (season_stats['season_xG'] >= min_xG) &  # Dynamic xG threshold
    (season_stats['form'] >= avg_form * 0.8) &  # 80% of average form
    (~season_stats['web_name'].isin(top_scorer_names))
]

# Replace position names
hidden_gems['position_name'] = hidden_gems['position_name'].replace({
    'Forward': 'FWD',
    'Midfielder': 'MID',
    'Defender': 'DEF',
    'Goalkeeper': 'GK'
})

hidden_gems_data = []

if len(hidden_gems) > 0:
    hidden_gems = hidden_gems.copy()

    # Define metrics for z-score calculation
    metrics = [
        'season_xG', 'season_xA', 'season_xCS', 'season_key_passes',
        'form', 'points_per_game', 'goals_per_game', 'assists_per_game',
        'points_per_million', 'minutes_per_game',
        'season_tackles', 'season_recoveries', 'defensive_contribution_sum'
    ]

    # Compute per-game metrics to normalize for playing time
    hidden_gems['xG_per_game'] = hidden_gems['season_xG'] / hidden_gems['games_played']
    hidden_gems['xA_per_game'] = hidden_gems['season_xA'] / hidden_gems['games_played']
    hidden_gems['xCS_per_game'] = hidden_gems['season_xCS'] / hidden_gems['games_played']

    # Create z-scores with suffix "_z"
    for col in metrics + ['xG_per_game', 'xA_per_game', 'xCS_per_game']:
        if col in hidden_gems.columns:
            mean, std = hidden_gems[col].mean(), hidden_gems[col].std(ddof=0)
            if std > 0:
                hidden_gems[f"{col}_z"] = (
                    (hidden_gems[col] - mean) / std
                ).clip(lower=-3, upper=3)  # Cap z-scores to prevent outliers
            else:
                hidden_gems[f"{col}_z"] = 0
        else:
            hidden_gems[f"{col}_z"] = 0

    # --- Potential score computation ---
    def calc_potential(row):
        pos = row['position_name']
        if pos == 'FWD':
            return (
                row['xG_per_game_z'] * 0.3 +
                row['xA_per_game_z'] * 0.2 +
                row['form_z'] * 0.35 +
                row['points_per_game_z'] * 0.15 +
                row['points_per_million_z'] * 0.1
            )
        elif pos == 'MID':
            return (
                row['xG_per_game_z'] * 0.25 +
                row['xA_per_game_z'] * 0.25 +
                row['form_z'] * 0.25 +
                row['season_key_passes_z'] * 0.15 +
                row['points_per_million_z'] * 0.1
            )
        elif pos == 'DEF':
            return (
                row['xCS_per_game_z'] * 0.3 +
                row['xA_per_game_z'] * 0.15 +
                row['form_z'] * 0.25 +
                row['defensive_contribution_sum_z'] * 0.2 +
                row['points_per_million_z'] * 0.1
            )
        elif pos == 'GK':
            return (
                row['xCS_per_game_z'] * 0.35 +
                row['form_z'] * 0.25 +
                row['points_per_game_z'] * 0.2 +
                row['points_per_million_z'] * 0.2
            )
        else:
            return 0

    hidden_gems['potential_score'] = hidden_gems.apply(calc_potential, axis=1)

    # Normalize potential score to [0, 10]
    min_score = hidden_gems['potential_score'].min()
    max_score = hidden_gems['potential_score'].max()
    if max_score != min_score:
        hidden_gems['potential_score'] = (
            (hidden_gems['potential_score'] - min_score) / (max_score - min_score) * 10
        )
    else:
        hidden_gems['potential_score'] = 5  # Handle edge case

    # Verify numeric output
    print("\nPotential score stats:")
    print(hidden_gems['potential_score'].describe())

    # Filter out players with very low potential scores
    hidden_gems_sorted = hidden_gems[hidden_gems['potential_score'] > 0].nlargest(12, 'potential_score')

    for i, (_, player) in enumerate(hidden_gems_sorted.iterrows(), 1):
        print(f"{i}. {player['web_name']} ({player['position_name']}, {player['team_name']} [{player['team_name_short']}])")
        print(f"   {player['season_points']:.0f} pts | {player['selected_by_percent']}% owned | £{player['now_cost']}m | Form: {player['form']}")
        print(f"   Potential Score: {player['potential_score']:.2f} | xG:{player['season_xG']:.2f}, xA:{player['season_xA']:.2f}, xCS:{player['season_xCS']:.2f}")

        hidden_gems_data.append({
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "position": player['position_name'],
            "points": int(player['season_points']),
            "ownership": player['selected_by_percent'],
            "price": player['now_cost'],
            "xG": round(player['season_xG'], 2),
            "xA": round(player['season_xA'], 2),
            "xCS": round(player['season_xCS'], 2),
            "form": player['form'],
            "potentialScore": round(player['potential_score'], 2)
        })

else:
    print("No hidden gems found with current criteria")

insights_data['hidden_gems'] = hidden_gems_data


# 4. GOAL SCORERS
print(f"\n⚽ GOAL SCORERS")
print("-" * 50)
goal_leaders = season_stats[season_stats['season_goals'] > 0].nlargest(12, 'season_goals')

goal_scorers_data = []
print("🥅 Top Goal Scorers:")
for i, (_, player) in enumerate(goal_leaders.iterrows(), 1):
    gpg = player['season_goals'] / player['games_played']
    player_data = {
        "player": player['web_name'],
        "team": player['team_name'],
        "team_short": player['team_name_short'],
        "goals": int(player['season_goals']),
        "goalsPerGame": round(gpg, 2),
        "points": int(player['season_points']),
        "price": player['now_cost'],
        "ownership": player['selected_by_percent'],
        "form": player['form']
    }
    goal_scorers_data.append(player_data)
    
    print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}]): {player['season_goals']:.0f} goals ({gpg:.2f}/game) | Form: {player['form']}")

insights_data['goal_scorers'] = goal_scorers_data

# 5. ASSIST PROVIDERS
assist_leaders = season_stats[season_stats['season_assists'] > 0].nlargest(12, 'season_assists')

assist_providers_data = []
print("\n🎯 Top Assist Providers:")
for i, (_, player) in enumerate(assist_leaders.iterrows(), 1):
    apg = player['season_assists'] / player['games_played']
    player_data = {
        "player": player['web_name'],
        "team": player['team_name'],
        "team_short": player['team_name_short'],
        "assists": int(player['season_assists']),
        "assistsPerGame": round(apg, 2),
        "points": int(player['season_points']),
        "price": player['now_cost'],
        "ownership": player['selected_by_percent'],
        "form": player['form']
    }
    assist_providers_data.append(player_data)
    
    print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}]): {player['season_assists']:.0f} assists ({apg:.2f}/game) | Form: {player['form']}")

insights_data['assist_providers'] = assist_providers_data

# 6. DEFENSIVE LEADERS
print(f"\n🛡️ DEFENSIVE LEADERS")
print("-" * 50)

defensive_candidates = season_stats[
    (season_stats['season_points'] >= 10) &
    (season_stats['games_played'] >= 3) &
    (season_stats['position_name'].isin(['Goalkeeper', 'Defender']))
].copy()

defensive_leaders_data = []
if len(defensive_candidates) > 0:
    defensive_candidates['defensive_score'] = (
        defensive_candidates['season_CS'] * 0.20 +  # Clean Sheets should be more heavily weighted
        defensive_candidates['season_tackles'] * 0.15 +  # Tackles are key for defensive performance
        defensive_candidates['season_recoveries'] * 0.15 +  # Recoveries are a crucial defensive stat
        defensive_candidates['season_xCS'] * 0.20 +  # xCS is predictive of future clean sheets, so it's very important
        defensive_candidates['defensive_contribution_sum'] * 0.10 +  # Overall defensive contributions
        defensive_candidates['clearances_blocks_interceptions_sum'] * 0.10 +  # Key to defensive stability
        (defensive_candidates['season_points'] / defensive_candidates['games_played']) * 0.10  # Points still matter, but not as much as the core defensive metrics
    )
    
    top_defenders = defensive_candidates.nlargest(12, 'defensive_score')
    print("🛡️ Best Defensive Performers:")
    for i, (_, player) in enumerate(top_defenders.iterrows(), 1):
        cs_rate = (player['season_CS'] / player['games_played']) * 100 if player['games_played'] > 0 else 0
        ppg = player['season_points'] / player['games_played'] if player['games_played'] > 0 else 0
        
        player_data = {
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "position": player['position_name'],
            "points": int(player['season_points']),
            "ppg": round(ppg, 1),
            "cleanSheets": int(player['season_CS']),
            "csRate": round(cs_rate, 1),
            "tackles": int(player['season_tackles']) if player['season_tackles'] > 0 else 1,
            "defensiveContributions": int(player['defensive_contribution_sum']) if 'defensive_contribution_sum' in player and player['defensive_contribution_sum'] > 0 else 1,
            "price": player['now_cost'],
            "form": player['form']
        }
        defensive_leaders_data.append(player_data)
        
        print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}], {player['position_name']})")
        print(f"     {player['season_points']:.0f} pts ({ppg:.1f} ppg) | {player['season_CS']:.0f} CS ({cs_rate:.1f}%) | {player['season_tackles']:.0f} tackles | £{player['now_cost']}m | Form: {player['form']}")

insights_data['defensive_leaders'] = defensive_leaders_data




# Export to JSON files
import json
import os

# Create output directory
output_dir = 'data/top_performers'
os.makedirs(output_dir, exist_ok=True)

# Export each category to separate JSON files
for category, data in insights_data.items():
    filename = f'{output_dir}/{category}.json'
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"\n✅ Exported {category}: {len(data)} players -> {filename}")

# Also create a combined file for convenience
combined_filename = f'{output_dir}/all_insights.json'
with open(combined_filename, 'w', encoding='utf-8') as f:
    json.dump(insights_data, f, indent=2, ensure_ascii=False)

print(f"\n🎉 ALL DATA EXPORTED TO JSON!")
print(f"📁 Output directory: {output_dir}/")
print(f"📊 Individual files: {list(insights_data.keys())}")
print(f"📦 Combined file: all_insights.json")

🏆 === FPL KEY INSIGHTS & RECOMMENDATIONS ===

⭐ TOP 15 SEASON PERFORMERS
--------------------------------------------------
 1. Haaland (Forward, Man City [MCI])
    70 pts (10.0 ppg) | £14.5m | 61.0% owned | Form: 9.9
 2. Semenyo (Midfielder, Bournemouth [BOU])
    66 pts (9.4 ppg) | £7.9m | 59.3% owned | Form: 9.0
 3. J.Timber (Defender, Arsenal [ARS])
    48 pts (6.9 ppg) | £5.9m | 17.6% owned | Form: 5.1
 4. Gabriel (Defender, Arsenal [ARS])
    47 pts (6.7 ppg) | £6.3m | 29.5% owned | Form: 8.8
 5. Guéhi (Defender, Crystal Palace [CRY])
    46 pts (6.6 ppg) | £4.8m | 30.8% owned | Form: 6.2
 6. Senesi (Defender, Bournemouth [BOU])
    46 pts (6.6 ppg) | £5.0m | 25.0% owned | Form: 6.6
 7. Caicedo (Midfielder, Chelsea [CHE])
    45 pts (6.4 ppg) | £5.8m | 14.0% owned | Form: 4.8
 8. Burn (Defender, Newcastle [NEW])
    43 pts (6.1 ppg) | £5.1m | 8.2% owned | Form: 7.3
 9. Calafiori (Defender, Arsenal [ARS])
    42 pts (6.0 ppg) | £5.7m | 14.8% owned | Form: 3.3
10. Anthony (Midfiel

C:\Users\kahme\AppData\Local\Temp\ipykernel_24788\1697044947.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hidden_gems['position_name'] = hidden_gems['position_name'].replace({


In [87]:
# 7. PERFORMANCE ANALYSIS
print(f"\n📈 OVERPERFORMANCE ANALYSIS")
print("-" * 50)

overperformers_data = []
sustainable_scorers_data = []
underperformers_data = []

if 'season_xG' in season_stats.columns and 'season_goals' in season_stats.columns:
    overperformance_candidates = season_stats[
        (season_stats['season_goals'] > 0) & 
        (season_stats['season_xG'] > 0) &
        (season_stats['games_played'] >= 3)
    ].copy()
    
    # Calculate overperformance and normalize by minutes played
    overperformance_candidates['goal_overperformance'] = overperformance_candidates['season_goals'] - overperformance_candidates['season_xG']
    if 'minutes_played' in season_stats.columns:
        overperformance_candidates['overperformance_per_90'] = overperformance_candidates['goal_overperformance'] / overperformance_candidates['minutes_played'] * 90
    else:
        overperformance_candidates['overperformance_per_90'] = overperformance_candidates['goal_overperformance'] / overperformance_candidates['games_played']

    # Dynamic threshold based on xG
    overperformance_candidates['threshold'] = 0.1 * overperformance_candidates['season_xG'].clip(lower=0.5)  # Minimum threshold of 0.5

    # Goal overperformers (regression risk)
    goal_overperformers = overperformance_candidates[
        overperformance_candidates['goal_overperformance'] > overperformance_candidates['threshold']
    ].nlargest(8, 'overperformance_per_90')
    
    print("⚡ Top Goal Overperformers (Potential Regression Risk):")
    for i, (_, player) in enumerate(goal_overperformers.iterrows(), 1):
        player_data = {
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "goals": int(player['season_goals']),
            "xG": round(player['season_xG'], 1),
            "overperformance": round(player['goal_overperformance'], 1),
            "overperformance_per_90": round(player['overperformance_per_90'], 3),
            "sustainable": False,
            "form": player['form']
        }
        overperformers_data.append(player_data)
        shots_info = f" | Shots: {player['shots']:.0f}, SoT: {player['shots_on_target']:.0f}" if 'shots' in player and 'shots_on_target' in player else ""
        print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}]): {player['season_goals']:.0f} goals vs {player['season_xG']:.1f} xG (+{player['goal_overperformance']:.1f}) | Per 90: {player['overperformance_per_90']:.3f} | Form: {player['form']}{shots_info}")

    # Sustainable scorers (goals close to xG)
    sustainable_scorers = overperformance_candidates[
        abs(overperformance_candidates['goal_overperformance']) <= overperformance_candidates['threshold']
    ].nlargest(8, 'season_goals')  # Sort by goals for relevance
    print("\n🌟 Sustainable Scorers (Consistent Performance):")
    for i, (_, player) in enumerate(sustainable_scorers.iterrows(), 1):
        player_data = {
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "goals": int(player['season_goals']),
            "xG": round(player['season_xG'], 1),
            "overperformance": round(player['goal_overperformance'], 1),
            "overperformance_per_90": round(player['overperformance_per_90'], 3),
            "sustainable": True,
            "form": player['form']
        }
        sustainable_scorers_data.append(player_data)
        shots_info = f" | Shots: {player['shots']:.0f}, SoT: {player['shots_on_target']:.0f}" if 'shots' in player and 'shots_on_target' in player else ""
        print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}]): {player['season_goals']:.0f} goals vs {player['season_xG']:.1f} xG ({player['goal_overperformance']:.1f}) | Per 90: {player['overperformance_per_90']:.3f} | Form: {player['form']}{shots_info}")

    # Underperformers (potential breakout candidates)
    goal_underperformers = overperformance_candidates[
        overperformance_candidates['goal_overperformance'] < -overperformance_candidates['threshold']
    ].nlargest(8, 'season_xG')  # Sort by xG for breakout potential
    print("\n🔥 Goal Underperformers (Potential Breakout Candidates):")
    for i, (_, player) in enumerate(goal_underperformers.iterrows(), 1):
        player_data = {
            "player": player['web_name'],
            "team": player['team_name'],
            "team_short": player['team_name_short'],
            "goals": int(player['season_goals']),
            "xG": round(player['season_xG'], 1),
            "overperformance": round(player['goal_overperformance'], 1),
            "overperformance_per_90": round(player['overperformance_per_90'], 3),
            "sustainable": False,
            "form": player['form']
        }
        underperformers_data.append(player_data)
        shots_info = f" | Shots: {player['shots']:.0f}, SoT: {player['shots_on_target']:.0f}" if 'shots' in player and 'shots_on_target' in player else ""
        print(f"  {i}. {player['web_name']} ({player['team_name']} [{player['team_name_short']}]): {player['season_goals']:.0f} goals vs {player['season_xG']:.1f} xG ({player['goal_overperformance']:.1f}) | Per 90: {player['overperformance_per_90']:.3f} | Form: {player['form']}{shots_info}")

    # Export to JSON
    os.makedirs('data/performance_analysis', exist_ok=True)
    with open('data/performance_analysis/overperformers.json', 'w', encoding='utf-8') as f:
        json.dump(overperformers_data, f, indent=4, ensure_ascii=False) 
    with open('data/performance_analysis/sustainable_scorers.json', 'w', encoding='utf-8') as f:
        json.dump(sustainable_scorers_data, f, indent=4 , ensure_ascii=False)
    with open('data/performance_analysis/underperformers.json', 'w', encoding='utf-8') as f:
        json.dump(underperformers_data, f, indent=4 , ensure_ascii=False)
    print("\nExported performance data to data/performance_analysis/")
else:
    print("❌ Missing required columns (season_xG or season_goals) for overperformance analysis")


📈 OVERPERFORMANCE ANALYSIS
--------------------------------------------------
⚡ Top Goal Overperformers (Potential Regression Risk):
  1. Caicedo (Chelsea [CHE]): 3 goals vs 0.6 xG (+2.4) | Per 90: 0.343 | Form: 4.8
  2. Semenyo (Bournemouth [BOU]): 6 goals vs 3.8 xG (+2.2) | Per 90: 0.314 | Form: 9.0
  3. Bowen (West Ham [WHU]): 3 goals vs 1.0 xG (+2.0) | Per 90: 0.286 | Form: 6.0
  4. Haaland (Man City [MCI]): 9 goals vs 7.0 xG (+2.0) | Per 90: 0.286 | Form: 9.9
  5. Woltemade (Newcastle [NEW]): 3 goals vs 1.9 xG (+1.1) | Per 90: 0.275 | Form: 4.5
  6. Isidor (Sunderland [SUN]): 3 goals vs 1.1 xG (+1.9) | Per 90: 0.271 | Form: 3.6
  7. Anthony (Burnley [BUR]): 4 goals vs 2.2 xG (+1.8) | Per 90: 0.257 | Form: 5.1
  8. Gravenberch (Liverpool [LIV]): 2 goals vs 0.3 xG (+1.7) | Per 90: 0.243 | Form: 7.5

🌟 Sustainable Scorers (Consistent Performance):
  1. Gyökeres (Arsenal [ARS]): 3 goals vs 3.0 xG (0.0) | Per 90: 0.000 | Form: 1.8
  2. Sarr (Crystal Palace [CRY]): 3 goals vs 2.9 xG (0

# 6️⃣ Strategic Analysis Tools {#strategic-analysis-tools}

## ⚔️ Advanced FPL Analysis Functions

This section contains powerful, reusable functions for Fantasy Premier League strategic analysis:

### 🔧 **Available Tools:**
1. **Defender Rankings** - Rank defenders by clean sheet potential and value
2. **Attacker Rankings** - Rank attacking players by goal/assist potential  
3. **Team Strength Analysis** - Calculate attacking and defensive strength for all teams
4. **Fixture Difficulty Calculator** - Score any specific matchup

### 📊 **Key Features:**
- Uses **cumulative season statistics** for accuracy
- Considers expected stats (xG, xA, xCS) for sustainability  
- Includes value scoring (points per £million)
- Accounts for consistency and minutes played
- Easily customizable parameters

# 7️⃣ Feature 2 Ranking Leaderboard


In [48]:
season_stats

,web_name,team_name,team_name_short,element_type,now_cost,selected_by_percent,season_points,season_minutes,season_goals,season_assists,...,season_recoveries,clearances_blocks_interceptions_sum,defensive_contribution_sum,position_name,points_per_million,points_per_game,minutes_per_game,goals_per_game,assists_per_game,form
0,A.Becker,Liverpool,LIV,1,5.4,8.0,20,540,0.0,0.0,...,51,12,0,Goalkeeper,3.70,2.86,77.14,0.00,0.00,1.6
1,A.García,Aston Villa,AVL,2,3.9,0.3,0,0,0.0,0.0,...,0,0,0,Defender,0.00,0.00,0.00,0.00,0.00,0.0
2,A.Jimenez,Bournemouth,BOU,2,4.5,0.0,1,149,0.0,0.0,...,7,4,11,Defender,0.22,0.25,37.25,0.00,0.00,0.4
3,A.Murphy,Newcastle,NEW,2,3.9,0.7,0,0,0.0,0.0,...,0,0,0,Defender,0.00,0.00,0.00,0.00,0.00,0.0
4,A.Ramsey,Burnley,BUR,3,4.4,2.8,0,0,0.0,0.0,...,0,0,0,Midfielder,0.00,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,Zubimendi,Arsenal,ARS,3,5.5,4.4,32,591,2.0,0.0,...,24,16,54,Midfielder,5.82,4.57,84.43,0.29,0.00,2.1
755,Álvarez,West Ham,WHU,3,4.9,0.1,0,0,0.0,0.0,...,0,0,0,Midfielder,0.00,0.00,0.00,0.00,0.00,0.0
756,Édouard,Crystal Palace,CRY,4,5.0,0.3,1,2,0.0,0.0,...,0,0,1,Forward,0.20,0.14,0.29,0.00,0.00,0.0
757,Ødegaard,Arsenal,ARS,3,7.9,1.9,13,201,0.0,1.0,...,10,6,19,Midfielder,1.65,1.86,28.71,0.00,0.14,1.5


In [49]:
import json

# 🏆 ENHANCED TEAM STRENGTH RANKINGS
print("="*70)
print("📊 COMPREHENSIVE TEAM STRENGTH RANKINGS")
print("="*70)
print("💡 Enhanced with all available defensive metrics for accurate fixture assessment")

def create_comprehensive_team_strength_rankings(season_data: pd.DataFrame) -> pd.DataFrame:
    """
    Create comprehensive team strength rankings using all available defensive metrics.
    Enhanced calculation includes tackles, recoveries, clearances, and expected stats.
    """
    
    # ⚽ ATTACKING STRENGTH CALCULATION
    attacking_stats = season_data.groupby('team_name').agg({
        'season_goals': 'sum',
        'season_xG': 'sum', 
        'season_assists': 'sum',
        'season_xA': 'sum',
        'season_shots': 'sum',
        'season_SoT': 'sum',
        'season_key_passes': 'sum',
        'games_played': 'mean'
    }).round(3)
    
    # 🛡️ COMPREHENSIVE DEFENSIVE STRENGTH CALCULATION
    # Include all defensive players (GK + DEF)
    defensive_players = season_data[season_data['element_type'].isin([1, 2])]
    
    if len(defensive_players) == 0:
        print("⚠️ Warning: No defensive players found in dataset")
        defensive_stats = pd.DataFrame(index=attacking_stats.index)
        # Set default values for missing defensive data
        default_values = {
            'season_CS': 3.0, 'season_xCS': 3.0, 'season_GC': 1.5, 'season_xGC': 1.5,
            'season_tackles': 15.0, 'season_recoveries': 20.0, 
            'season_CBI': 10.0, 'season_defensive_contribution': 5.0,
            'games_played': attacking_stats['games_played'].iloc[0] if len(attacking_stats) > 0 else 6
        }
        for col, val in default_values.items():
            defensive_stats[col] = val
    else:
        # Aggregate all available defensive metrics
        agg_dict = {
            'games_played': 'mean'
        }
        
        # Add available defensive columns
        defensive_columns = ['season_CS', 'season_xCS', 'season_GC', 'season_xGC',
                           'season_tackles', 'season_recoveries', 
                           'season_clearances_blocks_interceptions', 'season_defensive_contribution']
        
        for col in defensive_columns:
            if col in defensive_players.columns:
                agg_dict[col] = 'mean'
        
        defensive_stats = defensive_players.groupby('team_name').agg(agg_dict).round(3)
        
        # Rename long column name for easier handling
        if 'season_clearances_blocks_interceptions' in defensive_stats.columns:
            defensive_stats.rename(columns={'season_clearances_blocks_interceptions': 'season_CBI'}, inplace=True)
    
    # 📊 CALCULATE PER-GAME METRICS
    
    # Attacking per-game metrics
    attacking_stats['goals_pg'] = attacking_stats['season_goals'] / attacking_stats['games_played']
    attacking_stats['xG_pg'] = attacking_stats['season_xG'] / attacking_stats['games_played']
    attacking_stats['assists_pg'] = attacking_stats['season_assists'] / attacking_stats['games_played']
    attacking_stats['xA_pg'] = attacking_stats['season_xA'] / attacking_stats['games_played']
    attacking_stats['shots_pg'] = attacking_stats['season_shots'] / attacking_stats['games_played']
    attacking_stats['key_passes_pg'] = attacking_stats['season_key_passes'] / attacking_stats['games_played']
    
    # Defensive per-game metrics
    defensive_stats['CS_rate'] = defensive_stats['season_CS'] / defensive_stats['games_played']
    defensive_stats['xCS_rate'] = defensive_stats['season_xCS'] / defensive_stats['games_played']
    defensive_stats['GC_pg'] = defensive_stats['season_GC'] / defensive_stats['games_played']
    defensive_stats['xGC_pg'] = defensive_stats['season_xGC'] / defensive_stats['games_played']
 
    
    if 'season_tackles' in defensive_stats.columns:
        defensive_stats['tackles_pg'] = defensive_stats['season_tackles'] / defensive_stats['games_played']
    if 'season_recoveries' in defensive_stats.columns:
        defensive_stats['recoveries_pg'] = defensive_stats['season_recoveries'] / defensive_stats['games_played']
    if 'season_CBI' in defensive_stats.columns:
        defensive_stats['CBI_pg'] = defensive_stats['season_CBI'] / defensive_stats['games_played']
    if 'season_defensive_contribution' in defensive_stats.columns:
        defensive_stats['def_contrib_pg'] = defensive_stats['season_defensive_contribution'] / defensive_stats['games_played']
    
    # 🎯 ENHANCED STRENGTH CALCULATIONS
    
    # Attack Strength (weighted combination of multiple metrics)
    attacking_stats['attack_strength'] = (
        attacking_stats['xG_pg'] * 0.25 +           # Expected goals (predictive)
        attacking_stats['goals_pg'] * 0.20 +        # Actual goals (results)
        attacking_stats['xA_pg'] * 0.20 +           # Expected assists (creativity)
        attacking_stats['assists_pg'] * 0.15 +      # Actual assists
        attacking_stats['shots_pg'] * 0.10 +        # Shot volume
        attacking_stats['key_passes_pg'] * 0.10     # Key passes (creativity)
    )
    
    # Comprehensive Defense Strength (using all available metrics)
    defense_components = []
    weights = []
    
    # Core defensive metrics (always available)
    defense_components.append(defensive_stats['CS_rate'])
    weights.append(0.25)  # Clean sheet rate
    
    defense_components.append(1 / (defensive_stats['GC_pg'] + 0.1))
    weights.append(0.20)  # Goals conceded (inverted)
    
    # Expected metrics (if available)
    if 'xCS_rate' in defensive_stats.columns:
        defense_components.append(defensive_stats['xCS_rate'])
        weights.append(0.15)  # Expected clean sheet rate
    
    if 'xGC_pg' in defensive_stats.columns:
        defense_components.append(1 / (defensive_stats['xGC_pg'] + 0.1))
        weights.append(0.15)  # Expected goals conceded (inverted)
    
    # 🎯 IMPROVED: Defensive actions with min-max normalization (more balanced)
    if 'tackles_pg' in defensive_stats.columns:
        tackles_norm = defensive_stats['tackles_pg'] / defensive_stats['tackles_pg'].max() if defensive_stats['tackles_pg'].max() > 0 else defensive_stats['tackles_pg']
        defense_components.append(tackles_norm)
        weights.append(0.10)
    
    if 'recoveries_pg' in defensive_stats.columns:
        recoveries_norm = defensive_stats['recoveries_pg'] / defensive_stats['recoveries_pg'].max() if defensive_stats['recoveries_pg'].max() > 0 else defensive_stats['recoveries_pg']
        defense_components.append(recoveries_norm)
        weights.append(0.05)
    
    if 'CBI_pg' in defensive_stats.columns:
        cbi_norm = defensive_stats['CBI_pg'] / defensive_stats['CBI_pg'].max() if defensive_stats['CBI_pg'].max() > 0 else defensive_stats['CBI_pg']
        defense_components.append(cbi_norm)
        weights.append(0.05)
    
    if 'def_contrib_pg' in defensive_stats.columns:
        def_contrib_norm = defensive_stats['def_contrib_pg'] / defensive_stats['def_contrib_pg'].max() if defensive_stats['def_contrib_pg'].max() > 0 else defensive_stats['def_contrib_pg']
        defense_components.append(def_contrib_norm)
        weights.append(0.05)
    
    # Normalize weights to sum to 1
    total_weight = sum(weights)
    weights = [w/total_weight for w in weights]
    
    # Calculate weighted defensive strength
    defensive_stats['defense_strength'] = sum(comp * weight for comp, weight in zip(defense_components, weights))
    
    # 🏆 COMBINE TEAM RANKINGS
    team_rankings = attacking_stats[['attack_strength']].join(
        defensive_stats[['defense_strength']], how='outer'
    )
    
    # 🔧 FIXED: Handle missing data (pandas 3.0 compatible)
    team_rankings = team_rankings.fillna({
        'attack_strength': team_rankings['attack_strength'].median(),
        'defense_strength': team_rankings['defense_strength'].median()
    })
    
    # Overall strength calculation
    team_rankings['overall_strength'] = (
        team_rankings['attack_strength'] * 0.6 + 
        team_rankings['defense_strength'] * 0.4
    )
    
    # Generate rankings
    team_rankings['attack_rank'] = team_rankings['attack_strength'].rank(ascending=False, method='dense').astype(int)
    team_rankings['defense_rank'] = team_rankings['defense_strength'].rank(ascending=False, method='dense').astype(int)
    team_rankings['overall_rank'] = team_rankings['overall_strength'].rank(ascending=False, method='dense').astype(int)
    
    return team_rankings.round(3)

# Generate comprehensive team rankings
team_rankings = create_comprehensive_team_strength_rankings(season_stats)
team_rankings_sorted = team_rankings.sort_values('overall_rank')

print("🏆 COMPREHENSIVE TEAM STRENGTH RANKINGS")
print("=" * 65)
print("📋 All Teams Ranked (Enhanced with Complete Defensive Analysis):")
print(team_rankings_sorted[['overall_rank', 'attack_rank', 'defense_rank', 
                           'overall_strength', 'attack_strength', 'defense_strength']].to_string())

print(f"\n⚽ TOP ATTACKING TEAMS:")
attack_rankings = team_rankings.sort_values('attack_rank').head(20)
for idx, (team, data) in enumerate(attack_rankings.iterrows(), 1):
    team_short = season_stats[season_stats['team_name'] == team]['team_name_short'].iloc[0] if not season_stats[season_stats['team_name'] == team].empty else 'UNK'
    print(f" {int(data['attack_rank']):2d}. {team:<15} [{team_short}] (Attack: {data['attack_strength']:.3f})")

print(f"\n🛡️ TOP DEFENSIVE TEAMS:")
defense_rankings = team_rankings.sort_values('defense_rank').head(20)
for idx, (team, data) in enumerate(defense_rankings.iterrows(), 1):
    team_short = season_stats[season_stats['team_name'] == team]['team_name_short'].iloc[0] if not season_stats[season_stats['team_name'] == team].empty else 'UNK'
    print(f" {int(data['defense_rank']):2d}. {team:<15} [{team_short}] (Defense: {data['defense_strength']:.3f})")

# Fetch & Save to Json

os.makedirs('data/rankings', exist_ok=True)

# Update helper to include xG and goals conceded
def get_team_metrics(team_data_source):
    if len(team_data_source) > 0:
        games_played = team_data_source['games_played'].iloc[0]

        goals_pg = team_data_source['season_goals'].sum() / games_played if 'season_goals' in team_data_source else 0
        xg_pg = team_data_source['season_xG'].sum() / games_played if 'season_xG' in team_data_source else 0
        clean_sheet_rate = team_data_source['season_CS'].iloc[0] / games_played if 'season_CS' in team_data_source else 0
        goals_conceded_pg = team_data_source['season_GC'].iloc[0] / games_played if 'season_GC' in team_data_source else 0
        def_contributions = team_data_source['season_defensive_contribution'].iloc[0] if 'season_defensive_contribution' in team_data_source else 0
    else:
        goals_pg = xg_pg = clean_sheet_rate = goals_conceded_pg = def_contributions = 0.0

    return (
        round(goals_pg, 2),
        round(xg_pg, 2),
        round(clean_sheet_rate, 2),
        round(goals_conceded_pg, 2),
        round(def_contributions, 2)
    )

# Function to create team ranking data
def create_ranking_data(rankings, ranking_type):
    rankings_data = []
    for team, data in rankings.iterrows():
        team_data_source = season_stats[season_stats['team_name'] == team]

        # Updated unpacking
        goals_pg, xg_pg, clean_sheet_rate, goals_conceded_pg, def_contributions = get_team_metrics(team_data_source)

        team_short = team_data_source['team_name_short'].iloc[0] if not team_data_source.empty else 'UNK'
        
        team_data = {
            'team': team,
            'team_short': team_short,
            f'{ranking_type}_rank': int(data[f'{ranking_type}_rank']),
            'overall_rank': int(data['overall_rank']),
            'overall_strength': round(data['overall_strength'], 3),
            f'{ranking_type}_strength': round(data[f'{ranking_type}_strength'], 3),
            'goals_per_game': goals_pg,
            'expected_goals_per_game': xg_pg,
            'goals_conceded_per_game': goals_conceded_pg,
            'clean_sheet_rate': clean_sheet_rate,
            'defensive_contribution': def_contributions
        }
        rankings_data.append(team_data)
    return rankings_data



# Create and save Attack Rankings
attack_rankings = team_rankings.sort_values('attack_rank').head(20)
attack_rankings_data = create_ranking_data(attack_rankings, 'attack')

# Save attack rankings to JSON file
with open('data/rankings/attack_rankings.json', 'w') as f:
    json.dump(attack_rankings_data, f, indent=2)

# Create and save Defense Rankings
defense_rankings = team_rankings.sort_values('defense_rank').head(20)
defense_rankings_data = create_ranking_data(defense_rankings, 'defense')

# Save defense rankings to JSON file
with open('data/rankings/defense_rankings.json', 'w') as f:
    json.dump(defense_rankings_data, f, indent=2)
    

overall_rankings = team_rankings.sort_values('overall_rank').head(20)
overall_rankings_data = create_ranking_data(overall_rankings, 'overall')

# Save defense rankings to JSON file
with open('data/rankings/overall_rankings.json', 'w') as f:
    json.dump(overall_rankings_data, f, indent=2)




📊 COMPREHENSIVE TEAM STRENGTH RANKINGS
💡 Enhanced with all available defensive metrics for accurate fixture assessment
🏆 COMPREHENSIVE TEAM STRENGTH RANKINGS
📋 All Teams Ranked (Enhanced with Complete Defensive Analysis):
                overall_rank  attack_rank  defense_rank  overall_strength  attack_strength  defense_strength
team_name                                                                                                   
Arsenal                    1            2             1             2.777            3.950             1.018
Liverpool                  2            1            10             2.584            3.964             0.513
Man Utd                    3            3            14             2.503            3.892             0.420
Man City                   4            4             5             2.482            3.703             0.651
Chelsea                    5            5            12             2.331            3.562             0.483
Bournemouth    

In [31]:
# Initialize lists to avoid duplicates
attacking_picks = []
defensive_picks = []

def get_players_for_matchup(team, matchup_type, season_stats, team_rankings, n=3):
    team_players = season_stats[season_stats['team_name'] == team].copy()
    if team_players.empty:
        return pd.DataFrame()
    
    # Set defaults for missing columns
    default_cols = {
        'season_xG': 0.0, 'season_xGC': 0.0, 'season_CS': 0.0, 'season_xCS': 0.0,
        'season_points': 0.0, 'season_goals': 0.0, 'season_assists': 0.0,
        'season_xA': 0.0, 'season_shots': 0.0, 'season_SoT': 0.0, 'season_SiB': 0.0,
        'season_minutes': 0.0, 'now_cost': 5.0, 'selected_by_percent': 0.0, 'form': 0.0
    }
    for col, val in default_cols.items():
        if col not in team_players.columns:
            team_players[col] = val
    
    if 'games_played' not in team_players.columns or team_players['games_played'].sum() == 0:
        team_players['games_played'] = 1  # Fallback to avoid division by zero
    
    # Compute metrics
    team_players['points_per_game'] = team_players['season_points'] / team_players['games_played']
    team_players['points_per_million'] = team_players['season_points'] / team_players['now_cost'].replace(0, 1)
    team_players['consistency_score'] = np.minimum(team_players['season_minutes'] / team_players['games_played'] / 90, 1)
    
    if matchup_type == 'weak_defense':
        team_players['xg_per_game'] = team_players['season_xG'] / team_players['games_played']
        team_players['xa_per_game'] = team_players['season_xA'] / team_players['games_played']
        team_players['goals_per_game'] = team_players['season_goals'] / team_players['games_played']
        team_players['assists_per_game'] = team_players['season_assists'] / team_players['games_played']
        team_players['shots_per_game'] = team_players['season_shots'] / team_players['games_played']
        team_players['SoT_per_game'] = team_players['season_SoT'] / team_players['games_played']
        team_players['SiB_per_game'] = team_players['season_SiB'] / team_players['games_played']
        position_filter = team_players['position_name'].isin(['Forward', 'Midfielder'])
        # Composite attacker score
        team_players['attacker_score'] = (
            0.3 * team_players['xg_per_game'] +
            0.25 * team_players['xa_per_game'] +
            0.2 * team_players['goals_per_game'] +
            0.15 * team_players['assists_per_game'] +
            0.05 * team_players['SoT_per_game'] +
            0.05 * team_players['SiB_per_game']
        ) * 0.6 + 0.25 * team_players['points_per_million'] + 0.15 * team_players['consistency_score']
        sort_columns = ['attacker_score', 'points_per_game', 'xg_per_game']
        display_cols = [
            'web_name', 'position_name', 'now_cost', 'goals_per_game', 'assists_per_game',
            'xg_per_game', 'xa_per_game', 'shots_per_game', 'SoT_per_game', 'SiB_per_game',
            'points_per_game', 'points_per_million', 'consistency_score', 'selected_by_percent',
            'team_name_short', 'form'
        ]
    elif matchup_type == 'weak_attack':
        team_players['clean_sheet_rate'] = team_players['season_CS'] / team_players['games_played']
        team_players['xcs_per_game'] = team_players['season_xCS'] / team_players['games_played']
        team_players['xgc_per_game'] = team_players['season_xGC'] / team_players['games_played']
        team_players['goals_conceded_per_game'] = team_players['season_GC'] / team_players['games_played']
        position_filter = team_players['position_name'].isin(['Defender', 'Goalkeeper'])
        # Composite defender score
        team_players['defender_score'] = (
            0.4 * team_players['xcs_per_game'] +
            0.35 * team_players['clean_sheet_rate'] +
            0.15 / (team_players['goals_conceded_per_game'] + 0.1)
        ) * 0.6 + 0.25 * team_players['points_per_million'] + 0.15 * team_players['consistency_score']
        sort_columns = ['defender_score', 'clean_sheet_rate']
        display_cols = [
            'web_name', 'position_name', 'now_cost', 'clean_sheet_rate', 'xcs_per_game',
            'goals_conceded_per_game', 'points_per_game', 'points_per_million',
            'consistency_score', 'selected_by_percent', 'team_name_short', 'form'
        ]
    else:
        return pd.DataFrame()
    
    filtered_players = team_players[position_filter]
    if filtered_players.empty:
        return pd.DataFrame()
    
    for col in sort_columns:
        if col not in filtered_players.columns:
            filtered_players[col] = 0.0
    
    result = filtered_players.sort_values(by=sort_columns, ascending=False).head(n)[display_cols]
    return result.round(3)

# SHOW ALL TEAMS: Complete attacking rankings with player recommendations
print(f"\n⚽ ATTACKING PICKS FROM ALL TEAMS (Sorted by Attack Rank):")
print("=" * 60)
all_attacking_teams = team_rankings.sort_values('attack_rank').head(20)  # Limit to top 20 teams

for idx, (team, data) in enumerate(all_attacking_teams.iterrows()):
    if team in season_stats['team_name'].values:
        attack_rank = int(data['attack_rank'])
        attack_strength = data['attack_strength']
        overall_strength = data['overall_strength']
        
        attackers = get_players_for_matchup(team, 'weak_defense', season_stats, team_rankings, 3)
        if not attackers.empty:
            print(f"\n🔴 {team} (#{attack_rank} Attack, Strength: {attack_strength:.3f}, Overall: {overall_strength:.3f}):")
            print(attackers.to_string(index=False))
            
            # Collect for JSON
            team_data = {
                'team': team,
                'attack_rank': attack_rank,
                'attack_strength': attack_strength,
                'overall_strength': overall_strength,
                'players': attackers.to_dict(orient='records')
            }
            attacking_picks.append(team_data)
        else:
            print(f"\n🔴 {team} (#{attack_rank} Attack, Strength: {attack_strength:.3f}, Overall: {overall_strength:.3f}): No attacking players found")

# SHOW ALL TEAMS: Complete defensive rankings with player recommendations  
print(f"\n🛡️ DEFENSIVE PICKS FROM ALL TEAMS (Sorted by Defense Rank):")
print("=" * 60)

all_defensive_teams = team_rankings.sort_values('defense_rank').head(20)  # Limit to top 20 teams

for idx, (team, data) in enumerate(all_defensive_teams.iterrows()):
    if team in season_stats['team_name'].values:
        defense_rank = int(data['defense_rank'])
        defense_strength = data['defense_strength']
        overall_strength = data['overall_strength']
        
        defenders = get_players_for_matchup(team, 'weak_attack', season_stats, team_rankings, 3)
        if not defenders.empty:
            print(f"\n🔵 {team} (#{defense_rank} Defense, Strength: {defense_strength:.3f}, Overall: {overall_strength:.3f}):")
            print(defenders.to_string(index=False))
            
            # Collect for JSON
            team_data = {
                'team': team,
                'defense_rank': defense_rank,
                'defense_strength': defense_strength,
                'overall_strength': overall_strength,
                'players': defenders.to_dict(orient='records')
            }
            defensive_picks.append(team_data)
        else:
            print(f"\n🔵 {team} (#{defense_rank} Defense, Strength: {defense_strength:.3f}, Overall: {overall_strength:.3f}): No defensive players found")

# Debugging: Print number of teams
print(f"\nProcessed {len(attacking_picks)} attacking teams")
print(f"Processed {len(defensive_picks)} defensive teams")

# Export to JSON
os.makedirs('data/quick_picks', exist_ok=True)

with open('data/quick_picks/attackingpicks.json', 'w', encoding='utf-8') as f:
    json.dump(attacking_picks, f, indent=4, ensure_ascii=False)

with open('data/quick_picks/defensivepicks.json', 'w', encoding='utf-8') as f:
    json.dump(defensive_picks, f, indent=4, ensure_ascii=False)

print("\nExported attacking picks to data/quick_picks/attackingpicks.json")
print("Exported defensive picks to data/quick_picks/defensivepicks.json")


⚽ ATTACKING PICKS FROM ALL TEAMS (Sorted by Attack Rank):

🔴 Liverpool (#1 Attack, Strength: 3.964, Overall: 2.584):
   web_name position_name  now_cost  goals_per_game  assists_per_game  xg_per_game  xa_per_game  shots_per_game  SoT_per_game  SiB_per_game  points_per_game  points_per_million  consistency_score  selected_by_percent team_name_short  form
Gravenberch    Midfielder       5.7           0.286             0.143        0.043        0.071           1.429         0.571         0.286            5.286               6.491              0.857                  5.9             LIV   7.5
      Gakpo    Midfielder       7.5           0.286             0.286        0.186        0.300           2.286         0.571         1.571            4.714               4.400              0.881                 11.9             LIV   3.0
 Szoboszlai    Midfielder       6.5           0.143             0.000        0.157        0.186           2.286         0.429         0.857            3.857         

attack_strength = (
    0.25 * xG_pg +
    0.20 * goals_pg +
    0.20 * xA_pg +
    0.15 * assists_pg +
    0.10 * shots_pg +
    0.10 * key_passes_pg
)

CS_rate	0.25	More clean sheets = better defense

1 / (GC_pg + 0.1)	0.20	Fewer goals conceded = stronger defense

xCS_rate (optional)	0.15	Model-based estimate of clean sheets

1 / (xGC_pg + 0.1)	0.15	Expected goals conceded (lower is better)

tackles_pg	0.10	Normalized by max in dataset

recoveries_pg	0.05	Normalized

CBI_pg	0.05	Normalized

def_contrib_pg	0.05	Normalized



attack_rank	Rank attack_strength, descending	1 = best attacking team
defense_rank	Rank defense_strength, descending	1 = best defensive team
overall_rank	Rank overall_strength, descending	1 = strongest all-around team

# 🔮  FIXTURE ANALYZER - SEASON-WIDE ANALYSIS



In [32]:
team_rankings

,attack_strength,defense_strength,overall_strength,attack_rank,defense_rank,overall_rank
team_name,,,,,,
Arsenal,3.950,1.018,2.777,2,1,1
Aston Villa,2.723,0.495,1.832,13,11,13
Bournemouth,3.353,0.629,2.263,6,6,6
Brentford,2.271,0.390,1.518,18,16,18
Brighton,3.139,0.378,2.035,9,17,10
Burnley,2.015,0.316,1.336,20,20,20
Chelsea,3.562,0.483,2.331,5,12,5
Crystal Palace,3.287,0.711,2.257,7,3,7
Everton,3.167,0.517,2.107,8,9,8


In [83]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import os

class EnhancedFixtureAnalyzer:
    """
    Advanced fixture analysis system for complete season planning
    
    Features:
    - Season-wide fixture difficulty analysis
    - Visual heatmaps and charts
    - Strategic transfer timing recommendations
    - Position-specific insights
    - Team matchup intelligence
    """
    
    def __init__(self, season_stats, team_rankings, fixtures_path='fixture_template.csv'):
        """Initialize with your existing data"""
        self.season_stats = season_stats
        self.team_rankings = team_rankings
        self.fixtures_df = pd.read_csv(fixtures_path)
        self.current_gw = season_stats['last_gameweek'].max()
        self.start_gw = self.current_gw + 1
        self._process_data()
        
    def _process_data(self):
        """Process the data and create team mappings"""
        self._map_team_names()
    
    def _map_team_names(self):
        """Map fixture team names to season_stats team names"""
        fixture_teams = set(self.fixtures_df['home_team'].unique()) | set(self.fixtures_df['away_team'].unique())
        season_teams = set(self.season_stats['team_name'].unique())
        
        self.team_mapping = {}
        
        for fixture_team in fixture_teams:
            if fixture_team in season_teams:
                self.team_mapping[fixture_team] = fixture_team
                continue
            best_match = None
            for season_team in season_teams:
                if (fixture_team.lower().replace(' ', '') in season_team.lower().replace(' ', '') or
                    season_team.lower().replace(' ', '') in fixture_team.lower().replace(' ', '')):
                    best_match = season_team
                    break
            if best_match:
                self.team_mapping[fixture_team] = best_match
            else:
                self.team_mapping[fixture_team] = fixture_team
                print(f"⚠️ Could not match '{fixture_team}' - using default mapping")
                
    def get_fixture_difficulty_matrix(self, start_gw=None, end_gw=None, home_advantage=0.9):
        """Create fixture difficulty matrix using BASIC SYSTEM calculation method"""
        if start_gw is None:
            start_gw = self.fixtures_df['gameweek'].min()
        if end_gw is None:
            end_gw = self.fixtures_df['gameweek'].max()
            
        fixtures_period = self.fixtures_df[
            (self.fixtures_df['gameweek'] >= start_gw) & 
            (self.fixtures_df['gameweek'] <= end_gw)
        ].copy()
        
        difficulties = []
        total_teams = len(self.team_rankings)
        
        for _, fixture in fixtures_period.iterrows():
            home_team = self.team_mapping.get(fixture['home_team'], fixture['home_team'])
            away_team = self.team_mapping.get(fixture['away_team'], fixture['away_team'])
            
            if home_team in self.team_rankings.index and away_team in self.team_rankings.index:
                home_stats = self.team_rankings.loc[home_team]
                away_stats = self.team_rankings.loc[away_team]
                
                home_attack_rank = int(home_stats['attack_rank'])
                away_defense_rank = int(away_stats['defense_rank'])
                
                original_home_attack = home_attack_rank
                if home_advantage > 0 and home_attack_rank > 1:
                    home_attack_rank = max(1, home_attack_rank - home_advantage)
                
                attack_rank_difference = away_defense_rank - home_attack_rank
                attack_difficulty = attack_rank_difference / total_teams * 10
                
                home_defense_rank = int(home_stats['defense_rank'])
                away_attack_rank = int(away_stats['attack_rank'])
                
                original_home_defense = home_defense_rank
                if home_advantage > 0 and home_defense_rank > 1:
                    home_defense_rank = max(1, home_defense_rank - home_advantage)
                
                defense_rank_difference = away_attack_rank - home_defense_rank
                defense_difficulty = defense_rank_difference / total_teams * 10
                
                difficulties.append({
                    'gameweek': fixture['gameweek'],
                    'home_team': fixture['home_team'],
                    'away_team': fixture['away_team'],
                    'mapped_home': home_team,
                    'mapped_away': away_team,
                    'attack_difficulty': attack_difficulty,
                    'defense_difficulty': defense_difficulty,
                    'overall_difficulty': (attack_difficulty + defense_difficulty) / 2,
                    'home_attack_rank': home_attack_rank,
                    'away_defense_rank': away_defense_rank,
                    'home_defense_rank': home_defense_rank,
                    'away_attack_rank': away_attack_rank,
                    'attack_rank_diff': attack_rank_difference,
                    'defense_rank_diff': defense_rank_difference
                })
        
        return pd.DataFrame(difficulties)


    def export_fixture_data(self, num_gameweeks=3):
        """Export fixture data, opportunities, and team summaries to JSON for front-end"""
        # Helper function to get difficulty level
        def get_difficulty_text(score):
            if score >= 4.0: return "Very Easy"
            elif score >= 2.5: return "Easy"
            elif score >= 1.0: return "Medium-Easy"
            elif score >= -0.5: return "Medium"
            elif score >= -2.0: return "Hard"
            else: return "Very Hard"
        
        # Helper function to get team_name_short
        def get_team_short(team):
            mapped_team = self.team_mapping.get(team, team)
            team_data = self.season_stats[self.season_stats['team_name'] == mapped_team]
            return team_data['team_name_short'].iloc[0] if 'team_name_short' in team_data.columns and not team_data.empty else team
        
        # 1. Fixtures Data (all game weeks)
        start_gw = self.start_gw
        end_gw = start_gw
        difficulty_matrix = self.get_fixture_difficulty_matrix()  # All game weeks
        fixtures_data = []
        for _, fixture in difficulty_matrix.iterrows():
            home_team = fixture['home_team']
            away_team = fixture['away_team']
            gw = int(fixture['gameweek'])
            mapped_home = fixture['mapped_home']
            mapped_away = fixture['mapped_away']
            
            home_att_score = round(fixture['attack_difficulty'], 1)
            home_def_score = round(fixture['defense_difficulty'], 1)
            
            # Calculate away team scores
            away_att_score = 0.0
            away_def_score = 0.0
            if mapped_away in self.team_rankings.index and mapped_home in self.team_rankings.index:
                away_stats = self.team_rankings.loc[mapped_away]
                home_stats = self.team_rankings.loc[mapped_home]
                total_teams = len(self.team_rankings)
                
                away_attack_rank = int(away_stats['attack_rank'])
                home_defense_rank = int(home_stats['defense_rank'])
                away_att_rank_diff = home_defense_rank - away_attack_rank
                away_att_score = round(away_att_rank_diff / total_teams * 10, 1)
                
                away_defense_rank = int(away_stats['defense_rank'])
                home_attack_rank = int(home_stats['attack_rank'])
                away_def_rank_diff = home_attack_rank - away_defense_rank
                away_def_score = round(away_def_rank_diff / total_teams * 10, 1)
            
            home_total = home_att_score + home_def_score
            away_total = away_att_score + away_def_score
            favorability = home_team if home_total > away_total else away_team if away_total > home_total else "Neutral"
            
            fixture_data = {
                'gameweek': gw,
                'fixture': f"{home_team} vs {away_team}",
                'home_team': {
                    'name': home_team,
                    'short_name': get_team_short(home_team),
                    'attack': {'score': home_att_score, 'level': get_difficulty_text(home_att_score)},
                    'defense': {'score': home_def_score, 'level': get_difficulty_text(home_def_score)},
                    'rank': {
                        'attack': int(self.team_rankings.loc[mapped_home, 'attack_rank']) if mapped_home in self.team_rankings.index else 0,
                        'defense': int(self.team_rankings.loc[mapped_home, 'defense_rank']) if mapped_home in self.team_rankings.index else 0
                    }
                },
                'away_team': {
                    'name': away_team,
                    'short_name': get_team_short(away_team),
                    'attack': {'score': away_att_score, 'level': get_difficulty_text(away_att_score)},
                    'defense': {'score': away_def_score, 'level': get_difficulty_text(away_def_score)},
                    'rank': {
                        'attack': int(self.team_rankings.loc[mapped_away, 'attack_rank']) if mapped_away in self.team_rankings.index else 0,
                        'defense': int(self.team_rankings.loc[mapped_away, 'defense_rank']) if mapped_away in self.team_rankings.index else 0
                    }
                },
                'favorability': favorability
            }
            fixtures_data.append(fixture_data)
        
        # 2. Fixture Opportunities
        opportunities_data = {'attack': [], 'defense': []}

        # Filter fixtures for the next num_gameweeks
        start_gw = self.start_gw
        end_gw = start_gw + num_gameweeks - 1  # Cover exactly num_gameweeks
        relevant_fixtures = [f for f in fixtures_data if start_gw <= f['gameweek'] <= end_gw]

        if not relevant_fixtures:
            print(f"⚠️ No fixtures found for gameweeks {start_gw} to {end_gw}")
        else:
            for position_type in ['attack', 'defense']:
                all_opportunities = []
                
                for fixture in relevant_fixtures:
                    gw = fixture['gameweek']
                    home_team = fixture['home_team']['name']
                    away_team = fixture['away_team']['name']
                    
                    # Home team opportunity
                    all_opportunities.append({
                        'gameweek': gw,
                        'team': home_team,
                        'short_name': fixture['home_team']['short_name'],
                        'opponent': away_team,
                        'attack_score': fixture['home_team']['attack']['score'],
                        'defense_score': fixture['home_team']['defense']['score'],
                        'combined_score': round((fixture['home_team']['attack']['score'] + fixture['home_team']['defense']['score']) / 2, 1),
                        'venue': 'H'
                    })
                    
                    # Away team opportunity
                    all_opportunities.append({
                        'gameweek': gw,
                        'team': away_team,
                        'short_name': fixture['away_team']['short_name'],
                        'opponent': home_team,
                        'attack_score': fixture['away_team']['attack']['score'],
                        'defense_score': fixture['away_team']['defense']['score'],
                        'combined_score': round((fixture['away_team']['attack']['score'] + fixture['away_team']['defense']['score']) / 2, 1),
                        'venue': 'A'
                    })
                
                # Sort by attack_score for 'attack' type, defense_score for 'defense' type
                sort_key = 'attack_score' if position_type == 'attack' else 'defense_score'
                all_opportunities.sort(key=lambda x: x[sort_key], reverse=True)
                opportunities_data[position_type] = all_opportunities[:10]

        
        # 3. Team Fixture Summary (all game weeks)
        all_difficulties = self.get_fixture_difficulty_matrix(start_gw, end_gw)
        if all_difficulties.empty:
            print("❌ No fixture difficulty data available")
            return []

        team_summary = []
        fixture_teams = set(all_difficulties['home_team'].unique()) | set(all_difficulties['away_team'].unique())

        for team in fixture_teams:
            # Filter all fixtures for the team (home and away)
            team_fixtures = all_difficulties[(all_difficulties['home_team'] == team) | (all_difficulties['away_team'] == team)]
            if len(team_fixtures) == 0:
                continue

            attack_scores = []
            defense_scores = []
            favorable_fixtures = 0

            # Process each fixture
            for _, fixture in team_fixtures.iterrows():
                if fixture['home_team'] == team:
                    # Home fixture: use precomputed difficulties
                    attack_diff = fixture['attack_difficulty']
                    defense_diff = fixture['defense_difficulty']
                else:
                    # Away fixture: use precomputed difficulties
                    attack_diff = fixture['attack_difficulty']
                    defense_diff = fixture['defense_difficulty']

                attack_scores.append(attack_diff)
                defense_scores.append(defense_diff)

                # Count favorable fixtures
                if attack_diff >= 2.5 or defense_diff >= 2.5:
                    favorable_fixtures += 1

            if not attack_scores or not defense_scores:
                continue

            # Compute averages
            avg_attack_diff = round(np.mean(attack_scores), 3)
            avg_defense_diff = round(np.mean(defense_scores), 3)
            overall_diff = round((avg_attack_diff + avg_defense_diff) / 2, 3)

            team_summary.append({
                'team': team,
                'avg_attack_difficulty': avg_attack_diff,
                'avg_defense_difficulty': avg_defense_diff,
                'overall_difficulty': overall_diff,
                'num_favorable_fixtures': favorable_fixtures
            })
        
        summary_data = sorted(team_summary, key=lambda x: x['overall_difficulty'], reverse=True)
        # Save to JSON
        os.makedirs('data/fixture_analysis', exist_ok=True)
        
        with open('data/fixture_analysis/fixtures.json', 'w') as f:
            json.dump(fixtures_data, f, indent=4)
        with open('data/fixture_analysis/fixture_opportunities.json', 'w') as f:
            json.dump(opportunities_data, f, indent=4)
        with open('data/fixture_analysis/team_fixture_summary.json', 'w') as f:
            json.dump(summary_data, f, indent=4)
        
        print("\nExported fixture data to data/fixture_analysis/fixtures.json")
        print("Exported fixture opportunities to data/fixture_analysis/fixture_opportunities.json")
        print("Exported team fixture summary to data/fixture_analysis/team_fixture_summary.json")

# Initialization block
print("🔮 INITIALIZING ENHANCED FIXTURE ANALYZER...")
print("=" * 60)

try:
    analyzer = EnhancedFixtureAnalyzer(season_stats, team_rankings, 'fixture_template.csv')
    print("✅ Analyzer initialized successfully!")
    print(f"📊 Fixture data loaded: {len(analyzer.fixtures_df)} fixtures")
    print(f"📅 Gameweeks available: {analyzer.fixtures_df['gameweek'].min()} to {analyzer.fixtures_df['gameweek'].max()}")
    print(f"🏟️ Teams mapped: {len(analyzer.team_mapping)} teams")
    
    missing_mappings = [team for team, mapped in analyzer.team_mapping.items() 
                       if mapped not in analyzer.team_rankings.index and mapped == team]
    
    if missing_mappings:
        print(f"⚠️ Teams without ranking data: {', '.join(missing_mappings[:5])}")
        print("   (These teams will be skipped in analysis)")
    else:
        print("✅ All teams successfully mapped to ranking data")
    
    print("\n🎯 ENHANCED FIXTURE ANALYZER READY!")

    
except Exception as e:
    print(f"❌ Error initializing analyzer: {e}")
    print("Please check that 'fixture_template.csv' exists and has the correct format")
    import traceback
    traceback.print_exc()

🔮 INITIALIZING ENHANCED FIXTURE ANALYZER...
✅ Analyzer initialized successfully!
📊 Fixture data loaded: 80 fixtures
📅 Gameweeks available: 8 to 15
🏟️ Teams mapped: 20 teams
✅ All teams successfully mapped to ranking data

🎯 ENHANCED FIXTURE ANALYZER READY!


End == Checkpoint. Features Working


In [ ]:

analyzer.export_fixture_data()  # Export results to JSON


Exported fixture data to data/fixture_analysis/fixtures.json
Exported fixture opportunities to data/fixture_analysis/fixture_opportunities.json
Exported team fixture summary to data/fixture_analysis/team_fixture_summary.json
